### Importing the necessary libraries

In [9]:
from nltk.stem.lancaster import LancasterStemmer
import tensorflow
import tflearn
import random
import pickle
import numpy
import nltk
import json

stemmer = LancasterStemmer()

### Loading JSON Data

In [2]:
with open('intents.json') as file:
    data = json.load(file)

### Extracting Data

In [4]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

### Word Stemming

In [5]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

### Bag of Words

In [6]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

### Developing a Model

In [7]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Training & Saving the Model

In [8]:
model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 0.00395 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.00395 - acc: 1.0000 -- iter: 16/20
Training Step: 3000  | total loss: 0.00401 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.00401 - acc: 1.0000 -- iter: 20/20
--
INFO:tensorflow:F:\Data\Projects\Advanced Python\Chat Bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


### Loading the Model

In [10]:
with open("intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from F:\Data\Projects\Advanced Python\Chat Bot\model.tflearn


### Making Predictions

In [17]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Hi, I'm Chatterbox. Here to help you with your queries about our store. (type quit or bye to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            print('Thank you for visiting. Have a nice day!')
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print('Chatterbox: ' + random.choice(responses))

In [18]:
chat()

Hi, I'm Chatterbox. Here to help you with your queries about our store. (type quit or bye to stop)!
You: Hi
Chatterbox: Hello, thanks for visiting
You: what cards do you accept?
Chatterbox: We accept VISA, Mastercard and AMEX
You: ok
Chatterbox: Bye! Come back again soon.
You: bye
Chatterbox: Bye! Come back again soon.
You: quit
Thank you for visiting. Have a nice day!
